In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from io import StringIO

In [6]:
OUT_DIR = "/content/drive/MyDrive/실습용"
NAS_DIR = "/content/drive/MyDrive/실습용"
MODEL = "IO15_v070_erlo_unis_h0"

In [7]:
excel_path = os.path.join(OUT_DIR, "STATION/STN_INFO.xlsx")
Stn_data = pd.read_excel(excel_path)

# 열 선택, 이름 변경, 정렬
Stn_data = Stn_data[['지점', '지점명', '지점주소', '위도', '경도', '노장해발고도']]
Stn_data = Stn_data.rename(columns={'지점': 'stn_id', '지점명': 'stn_name',
                                    '지점주소': 'stn_addr', '위도': 'lat',
                                    '경도': 'lon', '노장해발고도': 'hgt'})
Stn_data = Stn_data.sort_values(by='stn_id')

In [8]:
# STN_ID
STN_ID = Stn_data['stn_id']

# VAR
VAR = pd.DataFrame({'stn_id': [0], 'var': [0.0], 'datetime': [""],
                    'fcsttime': [""], 'leadtime': [0]})

# s_data
s_data = pd.DataFrame(Stn_data.apply(lambda row: f"{row['stn_id']}/{row['lon']}/{row['lat']}",
                                     axis=1).str.split('/').tolist(),
                      columns=['stn_id', 'lon', 'lat'])

# Coordinate_log
Coordinate_log = pd.DataFrame({'log': [""]})

In [9]:
# 주어진 날짜 문자열을 datetime 객체로 변환
given_date_str = "2023-10-25"
given_date = datetime.strptime(given_date_str, "%Y-%m-%d")

In [ ]:
# 예시 날짜
given_date = datetime(2023, 10, 25)

# DATE_LIST 정의
DATE_LIST = given_date
# Check if the length of SRT is not 10
SRT = given_date.strftime("%Y%m%d") + "00"
if len(SRT) != 10:
    print("Error: Check the number of letters on the start and end days.")
    quit()  # Program quit

record_range = [1, 15, 16, 21]

VAR = pd.DataFrame(columns=['stn_id', 'var', 'datetime', 'fcsttime', 'leadtime'])

for HH in range(15, 39):
    filepath = os.path.join(NAS_DIR, f"{SRT}/{MODEL}{HH:02d}.2023102512.gb2")
    print(filepath)
    if os.path.exists(filepath):
        print(f"{SRT} leadtime: {HH}")

        for i in range(3, len(s_data), 3):
            for RNum in record_range:
                DATETIME = DATE_LIST + timedelta(hours=9)
                FCSTTIME = DATETIME + timedelta(hours=HH)

                # 파일 경로를 수정했어
                filepath = os.path.join(NAS_DIR, f"{DATE_LIST.strftime('%Y%m%d')}{MODEL}{HH:02d}.gb2")
                VAR = pd.DataFrame(columns=['stn_id', 'var', 'datetime', 'fcsttime', 'leadtime'])

        print(f"loading... ")
    else:
        print(f"########## {DATE_LIST} leadtime: {HH} is NA ##########")

print("-----------------------")
# 'spread data' and 'data preprocessing'
raw_data = VAR.iloc[:, [4, 2, 3, 0, 1]]
raw_data = raw_data.iloc[1:, :]
raw_data['group'] = pd.Series(['solar', 'u_ws', 'v_ws', 'temp']).repeat(len(raw_data) // 4)
result_data = raw_data.pivot(index='stn_id', columns='group', values='var').reset_index()

out_fcst = DATE_LIST.strftime('%Y%m%d') + '09'
out_made = (DATE_LIST + timedelta(days=1)).strftime('%Y%m%d') + '06'

OUT_PATH = os.path.join(OUT_DIR, "WEATHER", f"fcst_{out_fcst}_for24h_made_{out_made}_VAR.csv")

result_data.to_csv(OUT_PATH, sep=',', index=False, quotechar='"', quoting=0, mode='a')

print("########## UM-LDAPS data loading completed! ##########")


/content/drive/MyDrive/실습용/2023102500/l015_v070_erlo_unis_h015.2023102512.gb2
2023102500 leadtime: 15
loading... 
/content/drive/MyDrive/실습용/2023102500/l015_v070_erlo_unis_h016.2023102512.gb2
2023102500 leadtime: 16
loading... 
/content/drive/MyDrive/실습용/2023102500/l015_v070_erlo_unis_h017.2023102512.gb2
2023102500 leadtime: 17
loading... 
/content/drive/MyDrive/실습용/2023102500/l015_v070_erlo_unis_h018.2023102512.gb2
2023102500 leadtime: 18
loading... 
/content/drive/MyDrive/실습용/2023102500/l015_v070_erlo_unis_h019.2023102512.gb2
2023102500 leadtime: 19
loading... 
/content/drive/MyDrive/실습용/2023102500/l015_v070_erlo_unis_h020.2023102512.gb2
2023102500 leadtime: 20
loading... 
/content/drive/MyDrive/실습용/2023102500/l015_v070_erlo_unis_h021.2023102512.gb2
2023102500 leadtime: 21
loading... 
/content/drive/MyDrive/실습용/2023102500/l015_v070_erlo_unis_h022.2023102512.gb2
2023102500 leadtime: 22
loading... 
/content/drive/MyDrive/실습용/2023102

In [ ]:
# Load data
power_curve = pd.read_csv(os.path.join(OUT_DIR, "POWER_CURVE/wind_turbine_3000kw_power_curve.csv"))
rawdata = pd.read_csv(os.path.join('/content/drive/MyDrive/실습용/WEATHER', f"fcst_{out_made}_for24h_made_{out_fcst}_VAR.csv"), encoding="euc-kr")

# Load Region data
Region_solar_gen = pd.read_excel(os.path.join(OUT_DIR, "REGION_GEN/Region_gen.xlsx"), sheet_name="solar")
Region_wind_gen = pd.read_excel(os.path.join(OUT_DIR, "REGION_GEN/Region_gen.xlsx"), sheet_name="wind")

# Rename and select columns for Region data
Region_solar_gen = Region_solar_gen[['관측 번호', '설비용량', '지역구분', '지역번호']]
Region_solar_gen = Region_solar_gen.rename(columns={'관측 번호': 'stn_id', '설비용량': 'solar_cap', '지역번호': 'AREA'})

Region_wind_gen = Region_wind_gen[['관측 번호', '설비용량']]
Region_wind_gen = Region_wind_gen.rename(columns={'관측 번호': 'stn_id', '설비용량': 'wind_cap'})

# Rename columns in power_curve
power_curve = power_curve.rename(columns={'Cp': 'WT_Cp'})

# Merge dataframes
Poten_data = pd.merge(rawdata, Region_solar_gen, on='stn_id', how='left')
Poten_data = pd.merge(Poten_data, Region_wind_gen, on='stn_id', how='left')

# Rename and convert columns
Poten_data = Poten_data.rename(columns={'solar': 'direct_SR', 'temp': 'temp_air'})
Poten_data['temp_air'] = Poten_data['temp_air'] - 273.15

# Handle negative direct_SR values
Poten_data['direct_SR'] = Poten_data['direct_SR'].apply(lambda x: 0 if x < 0 else x)

# Solar panel module temperature calculation
Poten_data['solar_panel_temp'] = Poten_data['temp_air'] + ((45 - 20) / 800) * Poten_data['direct_SR']

# Convert datetime and fcsttime columns to datetime format
Poten_data['datetime'] = pd.to_datetime(Poten_data['datetime'])
Poten_data['fcsttime'] = pd.to_datetime(Poten_data['fcsttime'])


In [ ]:
# 태양광 에너지 잠재량 산출(단위:kW)
Poten_data['SD_EPotential'] = Poten_data['solar_cap'] * (
            Poten_data['direct_SR'] / 800) * (1 - (abs(alpha) * (Poten_data['solar_panel_temp'] - 20))) * beta

Poten_data['SD_EPotential'] = np.where(Poten_data['SD_EPotential'] < 0, 0, Poten_data['SD_EPotential'])

# 결과 데이터 생성
result_solar = Poten_data[['AREA', 'datetime', 'fcsttime', 'leadtime', 'SD_EPotential', 'solar_cap']].copy()
result_solar.rename(columns={"AREA": "AREA_CD", "datetime": "CRTN_TM", "fcsttime": "FCST_TM", "leadtime": "LEAD_TM",
                              "SD_EPotential": "FCST_EP", "solar_cap": "FCST_CAPA"}, inplace=True)
result_solar['PWR_EXC_TP_CD'] = 1
result_solar = result_solar[
    ['PWR_EXC_TP_CD', 'AREA_CD', 'CRTN_TM', 'FCST_TM', 'LEAD_TM', 'FCST_EP', 'FCST_CAPA']].dropna()

result_solar['AREA_CD'] = result_solar['AREA_CD'].replace({
    1: "강원도",
    2: "경기도",
    3: "경상남도",
    4: "경상북도",
    5: "전라남도",
    6: "전라북도",
    7: "충청남도",
    8: "충청북도",
    9: "제주도"
})

In [ ]:
import pandas as pd
import numpy as np
import math

# Constants
rho = 1.225  # Air density (kg/m^3)
pi = math.pi  # Pi value
r = 40  # Turbine radius (m)

# Solar energy potential calculation
Poten_data['SD_EPotential'] = Poten_data['solar_cap'] * (Poten_data['direct_SR'] / 800) * (1 - (np.abs(alpha) * (Poten_data['solar_panel_temp'] - 20))) * beta
Poten_data['SD_EPotential'] = np.where(Poten_data['SD_EPotential'] < 0, 0, Poten_data['SD_EPotential'])

# Select and rename columns for solar and wind results
result_solar = Poten_data[['AREA', 'datetime', 'fcsttime', 'leadtime', 'SD_EPotential', 'solar_cap']].copy()
result_solar = result_solar.rename(columns={'AREA': 'AREA_CD', 'datetime': 'CRTN_TM', 'fcsttime': 'FCST_TM', 'leadtime': 'LEAD_TM', 'SD_EPotential': 'FCST_EP', 'solar_cap': 'FCST_CAPA'})
result_solar['PWR_EXC_TP_CD'] = 1
result_solar = result_solar[['PWR_EXC_TP_CD', 'AREA_CD', 'CRTN_TM', 'FCST_TM', 'LEAD_TM', 'FCST_EP', 'FCST_CAPA']].dropna()
result_solar['AREA_CD'] = result_solar['AREA_CD'].replace({1: '강원도', 2: '경기도', 3: '경상남도', 4: '경상북도', 5: '전라남도', 6: '전라북도', 7: '충청남도', 8: '충청북도', 9: '제주도'})


# Save to CSV
result_solar.to_csv("/content/drive/MyDrive/실습용/result_solar.csv", index=False, encoding='cp949')
